In [1]:
using ForwardDiff
using LinearAlgebra
using Test
using Tensors

### SVK material 
strain energy:
$$\Phi(\textbf{E}) =( \frac{\lambda}{2}tr(\textbf{E})^2 + G  tr(\textbf{E}^2))$$
cosserat stress tensor:
$$\textbf{S}( \textbf{E} ) = \lambda tr(\textbf{E}) \textbf{I} + 2 G \textbf{E}$$
constitutive driver:
$$ \frac{\partial\textbf{S}}{\partial\textbf{E}} = \lambda tr(\textbf{E}) \textbf{I} + 2 G \textbf{E}$$




In [2]:
"Strain energy"
ϕ(𝔼::AbstractMatrix,λ=λ,G=G) = (λ / 2 * tr(𝔼)^2 + G * tr(𝔼^2))

"Cosserat stress tensor 𝕊."
function 𝕊_analytic(𝔼::AbstractMatrix,λ=λ,G=G) 
    S = λ * tr(𝔼) * Diagonal(ones(Float64, 3)) + 2 * G * 𝔼
    return S
end

𝕊_analytic_ij(𝔼::Matrix,i::Int,j::Int) = 𝕊_analytic(𝔼)[i,j]

"Aux function"
_vogit(𝕋::AbstractMatrix, α::Real=1) = [𝕋[1,1],𝕋[2,2],𝕋[3,3],α*𝕋[2,3], α*𝕋[1,3], α*𝕋[1,2]]


# S₁₁ = ForwardDiff.derivative()

_vogit

## ONSAS numerical results

In [3]:
λ = 0.5769
G = μ = 0.3846
# Green Lagrange

𝔼 = [0.18375  0.2925  0.51
    0.2925   0.435   0.72
    0.51     0.72    1.14]

𝔼_tensor = SymmetricTensor{2,3}(𝔼)

𝕊_test = 𝕊_analytic(𝔼)

# Constitutive driver SVK
∂𝕊∂𝔼_test = [
    1.3461  0.5769      0.5769      0.0       0.0       0.0 
    0.5769  1.3461      0.5769      0.0       0.0       0.0 
    0.5769  0.5769      1.3461      0.0       0.0       0.0 
     0.0       0.0       0.0      0.3846      0.0       0.0 
     0.0       0.0       0.0       0.0      0.3846      0.0 
     0.0       0.0       0.0       0.0        0.0      0.3846
     ]

6×6 Matrix{Float64}:
 1.3461  0.5769  0.5769  0.0     0.0     0.0
 0.5769  1.3461  0.5769  0.0     0.0     0.0
 0.5769  0.5769  1.3461  0.0     0.0     0.0
 0.0     0.0     0.0     0.3846  0.0     0.0
 0.0     0.0     0.0     0.0     0.3846  0.0
 0.0     0.0     0.0     0.0     0.0     0.3846

In [4]:
𝕊_test

3×3 Matrix{Float64}:
 1.15596   0.224991  0.392292
 0.224991  1.34922   0.553824
 0.392292  0.553824  1.89151

In [5]:
"Computes ∂S∂E with complex step."
function ∂𝕊∂𝔼_complex_step(𝕊::Function, λ::Real,G::Real , 𝔼::Matrix)

    compStep = 1e-10 ;
    ∂S∂E = zeros(6,6)

    for i in 1:6
  
      direction = zeros(3,3) ;
  
      if i<4
        direction[i,i] = 1 ;
      elseif i==4
        direction[2,3] = 1 ;
        direction[3,2] = 1 ;
      elseif i==5
        direction[1,3] = 1 ;
        direction[3,1] = 1 ;
      elseif i==6
        direction[1,2] = 1 ;
        direction[2,1] = 1 ;
      end
  
      𝔼_complex = 𝔼  + compStep * direction * im ;
  
      𝕊_complex = 𝕊(𝔼_complex, λ, G) ;
  
      dsde = _vogit( ( imag( 𝕊_complex ) / compStep ) , 1 ) ;
  
      ∂S∂E[1:3,i] = dsde[1:3] ;
      ∂S∂E[4:6,i] = dsde[4:6]*0.5 ;
    #   %~ ConsMat(:,i) = dsde ;
    end
    return ∂S∂E
  end


∂𝕊∂𝔼_ONSAS = ∂𝕊∂𝔼_complex_step(𝕊_analytic, λ, G, 𝔼)

@test ∂𝕊∂𝔼_test == ∂𝕊∂𝔼_ONSAS

Test Passed

## Forward diff for $\frac{\partial S}{\partial E} $

In [19]:
"Computes 𝕊 given a ϕ(𝔼) function."
𝕊_forward_diff(𝔼::AbstractMatrix, ϕ::Function=ϕ) = ForwardDiff.gradient(ϕ, 𝔼)

@test 𝕊_forward_diff(𝔼) == 𝕊_analytic(𝔼)

"Closure function returns 𝕊 (i,j)."
function 𝕊_forward_dif_ij(𝔼::AbstractMatrix, i::Int=i, j::Int=j, ϕ::Function = ϕ) 
    𝕊 = 𝕊_forward_diff(𝔼::AbstractMatrix, ϕ)
    return 𝕊[i,j]
end

∂S∂𝔼_forward_dif = zeros(6,6)

"Computes ∂S ∂E with analytic function for 𝕊ᵢⱼ ."
function ∂S∂𝔼_forward_diff(𝔼::Matrix, 𝕊_analytic::Function)

    indexes = [(1,1),(2,2),(3,3),(2,3),(1,3),(1,2)]

    ∂S∂𝔼_forward_diff = zeros(6,6)
    aux_gradients = zeros(3,3)

    row = 1
    for index in indexes
        i,j = index
        ∂S∂𝔼_forward_diff[row,:] .= _vogit(ForwardDiff.gradient!(aux_gradients,x -> 𝕊_analytic(x)[i,j], 𝔼),0.5)
        row+=1
    end
    return ∂S∂𝔼_forward_diff

end

∂𝕊∂𝔼_forward_diff =  ∂S∂𝔼_forward_diff(𝔼, 𝕊_analytic)
@test ∂𝕊∂𝔼_forward_diff ≈ ∂𝕊∂𝔼_test rtol = 1e-4

Test Passed

## Forward diff for $S$ and $ \frac{\partial S}{\partial E} $

In [10]:

"Computes ∂S ∂E with analytic function for ϕ."
function S_∂S∂𝔼_forward_diff(𝔼::Matrix, ϕ::Function=ϕ)
    
    𝕊 = 𝔼 -> ForwardDiff.gradient(ϕ, 𝔼)

    ∂S∂𝔼_forward_diff = zeros(6,6)
    aux_gradients = zeros(3,3)

    ∂S∂𝔼_forward_diff[1,:] .= _vogit(ForwardDiff.gradient!(aux_gradients,E -> 𝕊(E)[1,1], 𝔼),0.5)
    ∂S∂𝔼_forward_diff[2,:] .= _vogit(ForwardDiff.gradient!(aux_gradients,E -> 𝕊(E)[2,2], 𝔼),0.5)
    ∂S∂𝔼_forward_diff[3,:] .= _vogit(ForwardDiff.gradient!(aux_gradients,E -> 𝕊(E)[3,3], 𝔼),0.5)
    ∂S∂𝔼_forward_diff[4,:] .= _vogit(ForwardDiff.gradient!(aux_gradients,E -> 𝕊(E)[2,3], 𝔼),0.5)
    ∂S∂𝔼_forward_diff[5,:] .= _vogit(ForwardDiff.gradient!(aux_gradients,E -> 𝕊(E)[1,3], 𝔼),0.5)
    ∂S∂𝔼_forward_diff[6,:] .= _vogit(ForwardDiff.gradient!(aux_gradients,E -> 𝕊(E)[1,2], 𝔼),0.5)

    return 𝕊(𝔼), ∂S∂𝔼_forward_diff
end

S_forward_dif_from_ϕ, ∂S∂𝔼_forward_dif_from_ϕ = S_∂S∂𝔼_forward_diff(𝔼)

@test S_forward_dif_from_ϕ == 𝕊_test
@test ∂S∂𝔼_forward_dif_from_ϕ ≈ ∂𝕊∂𝔼_test rtol = 1e-4 skip = true

LoadError: LoadError: type Symbol has no field args
in expression starting at /home/mvanzulli/Repositories/gitHub/orgs/ONSAS/ONSAS.jl/src/Materials/Hyperelastic.ipynb:23

In [28]:

function constitutive_driver(E)
    # Compute all derivatives in one function call
    ∂²Ψ∂E², 𝕊 = Tensors.hessian(y -> ϕ(y), E, :all)
    # indexes = [(1,1),(2,2),(3,3),(3,2),(1,3),(1,2)]

    indexes = [(1,1),(2,2),(3,3),(2,3),(1,3),(1,2)]

    ∂S∂𝔼 = zeros(6,6)

    row = 1
    for index in indexes
        i,j = index
        ∂S∂𝔼[row,:] .= _vogit(∂²Ψ∂E²[:,:,i,j], 1)
        row+=1
    end
    

    return 𝕊, ∂S∂𝔼,∂²Ψ∂E²
end;

𝕊_from_ϕ_tensors, ∂S∂𝔼_tensors,∂²Ψ∂E² = constitutive_driver(𝔼_tensor)

@test 𝕊_from_ϕ_tensors == 𝕊_test
@test ∂S∂𝔼_tensors ≈ ∂𝕊∂𝔼_test rtol = 1e-4

Test Passed